# 3B.0

Today we are going upgrade our approach in a number of ways: We will combine some of the estimators we saw last time to get potentially better predictions. We will undersample our data to ameliorate some of the adverse effects of imbalanced data. And we will combine these two approach to retain information dispite undersampling and simultaneously quantify (types) of uncertainty.

First import numpy as np, pandas as pd, geopandas as gpd, matplotlib.pyplot as plt, pickle and time

We will also need a lot of stuff from sklearn. So from sklearn import metrics, from sklearn.neighbors import KNeighborsClassifier, from sklearn.naive_bayes import BernoulliNB and from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier and VotingClassifier. If in doubt look in week5/3A. The only new import is the VotingClassifier.

Now Load your pickles `train_set` and `val_set` **(Again: make sure you have run the week3correction.ipynb at some point first)**

You know the drill; make sure your data looks right in whatever way you like.

We will define a list of features and the X_train, y_train, X_val and y_val precisely as we did in week5/3A (So we will still just work with `binary_best_t1` as our sole target)

Now, lets take it one step att a time starting with ensemples

# 3B.1
## Introducing the concept of ensembles:

An machine learning ensample is 

"The idea of ensemble learning is to build a prediction model by combiningthe strengths of a collection of simpler base models. We havealready seen a number of examples that fall into this category.Bagging in Section 8.7 and random forests in Chapter 15 are ensemblemethods for classification, where a committee of trees each cast a vote forthe predicted class. Boosting in Chapter 10 was initially proposed as acommittee method as well, although unlike random forests, the committee of weak learners evolves over time, and the members cast a weighted vote. Stacking (Section 8.8) is a novel approach to combining the strengths ofa number of fitted models. In fact one could characterize any dictionarymethod, such as regression splines, as an ensemble method, with the basisfunctions serving the role of weak learners" (Friedman et al. 2001)

As you can see here you have already used ensambels: Random Forests and Adaboost! Crazy - it just happend behind the pretty curtains of Sklearn where we did not have to worry about it (except if you fiddled with the parameters). Today we will do a bit of the lifting ourselves. First, still with the help from Sklearn, than a bit more on our own.

Random Forest and Adaboost consisted of many of the same classifiers. But we can also combine different classifiers. For this porpurs we will use then Sklearn VotingClassifier (which you already loadet above). It is simply a framwork which allows more +1 models to "vote" on the final results. See these links for more info:

- https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html#sklearn.ensemble.VotingClassifier
- https://scikit-learn.org/stable/auto_examples/ensemble/plot_voting_decision_regions.html#sphx-glr-auto-examples-ensemble-plot-voting-decision-regions-py

Lets create a number of models amd fit the VotingClassifier:

In [ ]:
clf1 = BernoulliNB()
clf2 = AdaBoostClassifier(n_estimators=64, random_state = 0)
clf3 = RandomForestClassifier(n_estimators=32, n_jobs = -1, random_state = 0)
clf4 = KNeighborsClassifier(n_neighbors=3, n_jobs = -1)

eclf = VotingClassifier(estimators=[('NB', clf1), ('ADAB', clf2), ('RF', clf3), ('KNN', clf4)], voting='soft') # 'soft' so we get probabilities, not binaries
eclf = eclf.fit(X_train, y_train)

Use `eclf` to create predictions (`predict_proba`) for both `X_train` and `X_val` (see 3A for code on how-to)

Create and print the AUC, AP and Brier score regarding both you training set and validation set (Again, see 3A for how-to)

**Is how does this compare to the classifiers you used last time?**

Now, maybe some of our models are much better then others. But the bad models still offer som insights. So we want to hear the bad models, but we'll want to listen more closely to the good models. Luckly we can "weight" the models differently. That is, we can subscribe authority to some models over others. Say we might want to (for some reason) include logistic regression, but we do not want to hurt the over all performance of our model. We simply weigh it down compared to the other classifiers.  

Try giving the following weights to the new paramenter 'weights'

`[0.501,0.546,0.593,0.513]`


In [ ]:
Weclf = VotingClassifier(estimators=[('NB', clf1), ('ADAB', clf2), ('RF', clf3), ('KNN', clf4)], voting='soft', weights=...) # 'soft' so we get probabilities, not binaries
Weclf = Weclf.fit(X_train, y_train)

As you can see there is a weight for each model in the ensamble. So NB gets weighted as 0.501, Adaboost with 0.546 and so on.

**But why? Where did these number come from? They surely does not sum to one or anything.. (hint see 3A)**

You can choose any numbers you like: it is the relative difference between weights that counts. I have a particular reason for choosing the ones above but other valid reasons, convenitons and suluitons surely exist.   

Now,use `Weclf` to predict both `y_train_pred` and `y_val_pred`.

Create and print your new results:

So, perhaps not a big difference (at least not out-of-sample) but the weights are also pretty simmilar. Hopefully get the idea.

# 3B.2
## Tying together with imbalanced data and undersampling

Alright. So that was all about squeezing some more info out of our estimators. Now we are gonna take a leap back and talk about imbalanced data and then why the notion of ensamples is relevant for imbalanced data.   

As we know, when our data is highly imbalanced, the predictive algorithm will favor the classification of the majority class. In our case this would mean that the model(s) would be focused on predicting the absence of conflict and not the presence of conflict. A simple solution is to undersample the majority class. That is, in our train set we drop a portion of non-events to even the ratio between events and non-events. This is what is called Undersampling in the machine learning litterature. 

However, there are other types of undersampling, so to be more specific we denote it **Case-cohort sampling**. Case-cohort sampling implies, that we train our model(s) using all available events in the training set together with a randomly drawn and equally sized set of non-events also from the training set. This procedure is usually justified by arguing that more information is stored in the events than in non-events (See King and Zeng 2001).


But surely, you think, we most lose a lot of potential relevant information by discarding a huge chunck of our data - even if it is "only" some non-events. You are of course right and this is where ensambles enter agian - and things get a little complicated.. To amend the loss of information, we will also use a variant of **informed undersampling**. Informed undersampling implies that, instead of just using one model with one random subset of non-events, we use a large *ensemble* of models each using a new random subset of non-events (see He and Garcia 2008}. 

Specifically we can run some model (or indeed ensambel of models) $N$ times; each time with a new randomly drawn subset of non-events. As such we are effectively estimating a distribution of conflict probabilities for each cell thus taking advantage of all the information in events and non-events alike. We can then use the mean of these distributions as a maximun likelihood point estimate for the actual probability. 

Importantly, having a distribution of probabilities is in itself a huge benifit. It is indeed worth using this method simply to get this distribution as they allows us to infer how certain we are of the individual point estimates. If this does not make sense now, don't worry, it probably will soon enough.

I'll supply most, but not all, of the code here. Ofc feel to optimize, put into functions etc. Lets keep going with our weigthed voting ensamble `Weclf` from above. But now instead of running it one time we will run it multolpy times in a loop. Often will wil run this loop between 100 and 5000 times, but feel free to go as low as 25 here to save some time (`N_models = 25`). 

We can undersample in two ways on events or timelines. The conventional way as outlined above would be to simply keep events and sample non-events. An alternative approach given that we work with time series is to keep timelines (gids) that experienced conflcit at some point and sample timelines that never experienced conflict.  I have supplied the code for both appraoches below, but the timeline appraoches is set as default.


In [ ]:
Weclf = ...

np.random.RandomState(seed = 42)

# Udersampling, observation
# index_events = np.array(train_set[train_set['binary_best'] == 1].index)
# index_non_events = np.array(train_set[train_set['binary_best'] == 0].index)

# Undersampling, timelines
gid_events = train_set[train_set['binary_best'] == 1]['gid'].unique()
index_events = train_set[np.isin(train_set['gid'], gid_events, invert = False)].index
index_non_events = train_set[np.isin(train_set['gid'], gid_events, invert = True)].index

N_models = ...
N_events = index_events.shape[0]

random_index_non_events = np.random.choice(index_non_events, [N_models, N_events]) 


We will also need a "dictionary" to stor the results from each of the $N$ number of models

In [ ]:
dict_of_metrics = {}
dict_of_metrics['train_auc'] = []
dict_of_metrics['train_ap'] = []
dict_of_metrics['train_bs'] = []
dict_of_metrics['train_precision'] = []
dict_of_metrics['train_recall'] = []

dict_of_metrics['val_auc'] = []
dict_of_metrics['val_ap'] = []
dict_of_metrics['val_bs'] = []
dict_of_metrics['val_precision'] = []
dict_of_metrics['val_recall'] = []


Now we are ready to run 25 of our models! No code to change here, but I do recommend you put the code into a function if you are up for it. This will take some time, but each individual model run slightly faster now since there a less samples in any one models.

In [ ]:
start_time = time.time()

for i in np.arange(N_models):
    undersample_index = np.concatenate((random_index_non_events[i], index_events), axis = 0)
    
    X_train_US = X_train.iloc[undersample_index]
    y_train_US = y_train.iloc[undersample_index]
    
    Weclf = Weclf.fit(X_train_US, y_train_US)
    
    y_train_pred = Weclf.predict_proba(X_train_US)[:,1]
    y_val_pred = Weclf.predict_proba(X_val)[:,1]
    
    AUC_train = metrics.roc_auc_score(y_train_US, y_train_pred)
    AP_train = metrics.average_precision_score(y_train_US, y_train_pred)
    BS_train = metrics.brier_score_loss(y_train_US, y_train_pred)
    precision_train, recall_train, _ = metrics.precision_recall_curve(y_train_US, y_train_pred) 

    AUC_val = metrics.roc_auc_score(y_val, y_val_pred)
    AP_val = metrics.average_precision_score(y_val, y_val_pred)
    BS_val = metrics.brier_score_loss(y_val, y_val_pred)
    precision_val, recall_val, _ = metrics.precision_recall_curve(y_val, y_val_pred) 
    
    dict_of_metrics['train_auc'].append(AUC_train)
    dict_of_metrics['train_ap'].append(AP_train)
    dict_of_metrics['train_bs'].append(BS_train)
    dict_of_metrics['train_precision'].append(precision_train)
    dict_of_metrics['train_recall'].append(recall_train)


    dict_of_metrics['val_auc'].append(AUC_val)
    dict_of_metrics['val_ap'].append(AP_val)
    dict_of_metrics['val_bs'].append(BS_val)
    dict_of_metrics['val_precision'].append(precision_val)
    dict_of_metrics['val_recall'].append(recall_val)
    
    print(f'model {i + 1} out of {N_models} done.....', end='\r')
    
end_time = time.time()
run_time = (end_time - start_time)/60
    
print(f'Done! Runtime: {run_time:.2f} minutes')

Now, as mentioned we can just use the mean as a maximum likelyhood estimate:

In [ ]:
AUC_train = np.mean(dict_of_metrics['train_auc'])
AP_train = np.mean(dict_of_metrics['train_ap'])
BS_train = np.mean(dict_of_metrics['train_bs'])

AUC_val = np.mean(dict_of_metrics['val_auc'])
AP_val = np.mean(dict_of_metrics['val_ap'])
BS_val = np.mean(dict_of_metrics['val_bs'])

And print it as you usually do:

In [ ]:
print(f'\tTrain\tVal\nAUC: \t{AUC_train:.3f}\t{AUC_val:.3f}\nAP: \t{AP_train:.3f}\t{AP_val:.3f}\nBS: \t{BS_train:.3f}\t{BS_val:.3f}')

Well in this case you might not have done better then before. Maybe close to, maybe a bit worse. It all depends on the problem we have. But a perk of this is that we now can plot the a distribution!

# 3B.3
## Plots

Run the following code to generate some plots:

In [ ]:
plt.figure(figsize=[13,5])
plt.hist(dict_of_metrics['val_ap'])
plt.show()

plt.figure(figsize=[13,5])
plt.hist(dict_of_metrics['train_ap'])
plt.show()

In [ ]:

plt.title('PR curve')
plt.ylabel('precision')
plt.xlabel('recall')


for i in np.arange(N_models):
    
    val_recall = dict_of_metrics['val_recall'][i]
    val_precision = dict_of_metrics['val_precision'][i]

    plt.plot(val_recall, val_precision, color = 'blue', alpha = 0.2)
    
        
    train_recall = dict_of_metrics['train_recall'][i]
    train_precision = dict_of_metrics['train_precision'][i]

    plt.plot(train_recall, train_precision, color = 'orange', alpha = 0.2)
    
plt.show()

Naturally, the more models you run the better distribution you will get

**What is du you take from the plots above?**
   
**What does the difference between train and validation tell you?**

# 3B.4
## Bayesian correction


Now, one issue with this undersampling approach we used appoave is, that the specific probabilities produced will be somewhat inflated. All models are trained to believe that conflict is more common than it is. This can easily be amended using a *Bayesian prior correction* akin to what is presented in King and Zeng (2001a and 2001b) and implemented in Goldstone et al. (2010). 

Now it gets a bit mathy: we will use the overall probability of conflict in the last observed year in our training set: 2012. This is simply the ratio between events and non-events. Lets denote the share of events $Pr(E_{2012})$ and the share of non-events $Pr(NE_{2012})$. Then, denoting the estimated probabilities of an event in a specific cell at a specific year $Pr(E_{predicted})$. We denote the corresponding estimated probabilities of a non-event $Pr(NE_{predicted})$ and the corrected probabilities of events as $Pr(E_{corrected})$. Then the correction can be expressed as follows:

$$
Pr(E_{corrected}) = \frac{Pr(E_{predicted}) \times Pr(E_{2012})}{Pr(E_{predicted}) \times Pr(E_{2012})+Pr(NE_{predicted} \times Pr(NE_{2012})}
$$

If you take your time to look at the math and read the description above it is not as scary as it looks at first sight. It is even less scary to impliment.

In [ ]:
start_time = time.time()

for i in np.arange(N_models):
    undersample_index = np.concatenate((random_index_non_events[i], index_events), axis = 0)
    
    X_train_US = X_train.iloc[undersample_index]
    y_train_US = y_train.iloc[undersample_index]
    
    Weclf = Weclf.fit(X_train_US, y_train_US)
    
    y_train_pred = Weclf.predict_proba(X_train_US) # Notice here I now need both the prob of conflict and the prob of not conflict - even if they are just inverse
    y_val_pred = Weclf.predict_proba(X_val) # Notice here I now need both the prob of conflict and the prob of not conflict - even if they are just inverse
    
    
    # Baysian correction -----------------------------------------------------------------------------------

    Pr_E_2012 = X_train[X_train['year'] == 2012]['binary_best'].sum()/X_train[X_train['year'] == 2012].shape[0]
    
    Pr_NE_2012 = 1 - Pr_E_2012
    
    # needs for both train and val:
    Pr_E_train_pred = y_train_pred[:,1]
    Pr_NE_train_pred = y_train_pred[:,0] # could also just take the inverse 1-Pr_E_pred
    
    Pr_E_val_pred = y_val_pred[:,1]
    Pr_NE_val_pred = y_val_pred[:,0] # could also just take the inverse 1-Pr_E_pred
  
    y_train_pred_bayes = (Pr_E_train_pred*Pr_E_2012)/((Pr_E_train_pred*Pr_E_2012)+(Pr_NE_train_pred*Pr_NE_2012)) # this is just the math above.
    y_val_pred_bayes = (Pr_E_val_pred*Pr_E_2012)/((Pr_E_val_pred*Pr_E_2012)+(Pr_NE_val_pred*Pr_NE_2012)) # this is just the math above.
    
    # Baysian correction -----------------------------------------------------------------------------------

    
    AUC_train = metrics.roc_auc_score(y_train_US, y_train_pred_bayes)
    AP_train = metrics.average_precision_score(y_train_US, y_train_pred_bayes)
    BS_train = metrics.brier_score_loss(y_train_US, y_train_pred_bayes)
    precision_train, recall_train, _ = metrics.precision_recall_curve(y_train_US, y_train_pred_bayes) 

    AUC_val = metrics.roc_auc_score(y_val, y_val_pred_bayes)
    AP_val = metrics.average_precision_score(y_val, y_val_pred_bayes)
    BS_val = metrics.brier_score_loss(y_val, y_val_pred_bayes)
    precision_val, recall_val, _ = metrics.precision_recall_curve(y_val, y_val_pred_bayes) 
    
    dict_of_metrics['train_auc'].append(AUC_train)
    dict_of_metrics['train_ap'].append(AP_train)
    dict_of_metrics['train_bs'].append(BS_train)
    dict_of_metrics['train_precision'].append(precision_train)
    dict_of_metrics['train_recall'].append(recall_train)


    dict_of_metrics['val_auc'].append(AUC_val)
    dict_of_metrics['val_ap'].append(AP_val)
    dict_of_metrics['val_bs'].append(BS_val)
    dict_of_metrics['val_precision'].append(precision_val)
    dict_of_metrics['val_recall'].append(recall_val)
    
    print(f'model {i + 1} out of {N_models} done.....', end='\r')
    
end_time = time.time()
run_time = (end_time - start_time)/60
    
print(f'Done! Runtime: {run_time:.2f} minutes')

Create all the point estimates again (which is the mean of `dict_of_metrics['train_auc']`, `dict_of_metrics['train_ap']`, etc  - see above) 

In [ ]:
AUC_train = ...
AP_train = ...
BS_train = ...

AUC_val = ...
AP_val = ...
BS_val = ...

And print the new results:

Well, the difference is not striking here. Somes times it makes a difference and sometimes less so. It depends alot on the approach you take. Indeed the main point is not the get better results but to get "appropiate probabilities". That is - from a probability theori perspective - the Baysian correction also gives you more "correct" probabilities than the raw probabilities gotten through undersampling. This is because we here correct for the fact the to model see disproportionate amounts of conflcits compared to non-conflcits. As such, you can also use a a correction it if you are looking at a subset of the world (in which case it might also be appropriate). Now you know.  

Now, there are alot of things here done out of simplicity, which we could change. For instance I could have saved the actual probabilities, the outputs for a ROC-curve, and what not. In practise, when you rund these kind of model loops you want to take as much information as possible (almost) with you for further analysis - you might get a good idea later that you do noot have now and then it is irretating run run the loop again: esapcially if it is 1000 or 5000 models wih is often the case. I just railed myself in above for the sake of simplicity. (feel free to add/save other metrics - just remember to create a list in the dictionary for said metric first). Futhermore I allows "sample with replacement" and I do not care the loop samples the same gids across years or not. One could experiment with such appraoches - but these last points does not make sense to you don't worry about it :) 

# 3B.5
## Plots part two
Recreate (and maybe prettify) the plots from above but now with data from the new "corrected model":


# 3B.6
## Where's the Roc?

Now, given the undersample ensamble, try to see if you can save and extract the elements nessecary to create the corrospoding distibution of roc curves (and a mean point estimate). You can do it the same way we now extract the elements nessecary to create the PR-curve now. Things to not: See weel4/2B. Remember to make space in the dictionary for the nessecary elements. I konw this might be a bit hard for some, but do give it an honest try. It is easier than you think :) 



# 3B.7
## Quantifying (kinds of) Uncertainty

Read the first link and browes the next two:

- https://criticaluncertainties.com/2009/10/11/epistemic-and-aleatory-risk/

- https://en.wikipedia.org/wiki/Uncertainty_quantification

- http://apppm.man.dtu.dk/index.php/Epistemic_vs._Aleatory_uncertainty

**What uncertenties -- if any -- do we capture with our models?**  
**Can we capture other uncertainties? How?**  
**Does aleatory uncertainty really exist? Why why not?**

Please be a bit elaborate here - it is totally fine to go out on a limp and try to think big thoughts. uncertainty is a pretty abstract topic after, and you could spend you whole academic life on it (indeed many do). 

If you by chance are on the masters in Security Risk Management I particularly hope you will indulge me and spend a bit of time pondering these questions - or indeed ask some of your own.  

For to the last question maybe see inspiration maybe see https://en.wikipedia.org/wiki/Laplace%27s_demon

# 3B.8 (Optinal)

Run the loop again, but this time try to predict all targets binary_best_t1, binary_best_t2 etc. You will need to train $N$ different models for each target so you might need a loop within a loop: looping over each target, then over the random undersample indecies.  